In [1]:
import numpy as np
from scipy.optimize import minimize

# 1. Enter the Coefficients from your Level 3 Output
# (Hardcoding them here for clarity, in a real app we'd load the model)
coef_tv = 0.587
coef_social = 1.385
coef_radio = 4.67
# We ignore 'Week' for the optimization because we can't control time!

def predict_sales(budget_allocation):
    """
    Predicts sales based on a specific budget split.
    budget_allocation: A list of 3 numbers [TV, Social, Radio]
    """
    tv_spend, social_spend, radio_spend = budget_allocation
    
    # Note: We use a simplified formula here (Linear) for the optimizer
    # In a pro version, we would include the Saturation curve math here.
    predicted_sales = (coef_tv * tv_spend) + \
                      (coef_social * social_spend) + \
                      (coef_radio * radio_spend)
                      
    return predicted_sales

In [2]:
def objective_function(budget_allocation):
    """
    The function we want to minimize.
    We return negative sales because scipy minimizes.
    """
    sales = predict_sales(budget_allocation)
    return -sales # Negative Sales

In [3]:
total_budget = 50000 # The CMO gives you $50k

# Constraint: Sum of spends must equal total_budget
# format: {'type': 'eq', 'fun': lambda x: sum(x) - total_budget}
# (This means: sum(x) - 50000 must equal 0)
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - total_budget})

# Bounds: Each channel must be between $0 and $50000
bounds = ((0, total_budget), (0, total_budget), (0, total_budget))

In [4]:
# Initial Guess (We start by splitting it equally)
initial_guess = [total_budget/3, total_budget/3, total_budget/3]

# Run the Optimizer
result = minimize(
    objective_function, 
    initial_guess, 
    method='SLSQP', # Sequential Least Squares Programming (Standard for this)
    bounds=bounds, 
    constraints=constraints
)

print("Optimization Status:", result.success)
print(f"Max Predicted Sales: ${-result.fun:.2f}") # Flip the negative sign back!

print("\n--- OPTIMAL BUDGET ALLOCATION ---")
print(f"TV Spend:     ${result.x[0]:.2f}")
print(f"Social Spend: ${result.x[1]:.2f}")
print(f"Radio Spend:  ${result.x[2]:.2f}")

Optimization Status: True
Max Predicted Sales: $233500.00

--- OPTIMAL BUDGET ALLOCATION ---
TV Spend:     $0.00
Social Spend: $0.00
Radio Spend:  $50000.00
